# Current Population Survey Dashboard

### Data generation from bd CPS

-----

January 19, 2019

Brian W. Dew (brian.w.dew@gmail.com; @bd_econ)

-----

To Do:

- Table of values for various age groups (16-24, 25-34, 35-44, 45-54, 55-64, 65+)
- Next chart - prime age epop by state (change?)

In [8]:
import pandas as pd
import numpy as np
import os

os.chdir('/home/brian/Documents/CPS/data/clean/')

path = '/home/brian/Documents/CPS-dashboard/'

def write_txt(filename, filetext):
    """Write label to txt file"""
    with open(filename, 'w') as text_file:
        text_file.write(filetext)
        
def epop(group, variable, weight):
    """Calculate epop for group"""
    d = group[variable]
    w = group[weight]
    return (d * w).sum() / w.sum() * 100.0

year_range = range(1994, 2019)

#### EPOP line chart

In [2]:
data = pd.Series()
for year in year_range:
    df = (pd.read_feather(f'cps{year}.ft')
            .filter(['HRMONTH', 'AGE', 'BASICWGT', 'EMP'])
            .query('25 <= AGE <= 54')
            .groupby('HRMONTH')
            .apply(epop, 'EMP', 'BASICWGT'))
    df.index = pd.to_datetime([f'{year}-{i}-01' for i in df.index])
    data = data.append(df)

In [3]:
final = data.rolling(12).mean().dropna()

final.to_csv(f'{path}epop.csv', index_label='DATE', header=['epop'])

In [19]:
date = final.index[-1].strftime('%Y-%m')
val = round(final[-1], 1)
text = f'\scriptsize{{{date}}}\\\ \\footnotesize{{\\textbf{{{val}\%}}}}'
write_txt(f'{path}epop.txt', text)

#### EPOP text block

Need: Latest month, latest value, how many more/less would be employed at max and min.

In [16]:
# Get average 25-54 population in last twelve months
data = pd.Series()
for year in year_range[-2:]:
    df = (pd.read_feather(f'cps{year}.ft')
            .filter(['HRMONTH', 'AGE', 'BASICWGT'])
            .query('25 <= AGE <= 54')
            .groupby('HRMONTH')['BASICWGT'].sum())
    df.index = pd.to_datetime([f'{year}-{i}-01' for i in df.index])
    data = data.append(df)

In [38]:
latest_date = final.index[-1].strftime('%B %Y')
latest_val = final[-1] / 100.0
max_val = final.max() / 100.0
min_val = final.min() / 100.0
pop = data[-13:].mean() / 1_000_000.0

more_emp = round((max_val * pop) - (latest_val * pop), 1)
less_emp = round((latest_val * pop) - (min_val * pop), 1)

epop_text = f'{latest_date}, {str(round(latest_val * 100, 1))}'

write_txt(f'{path}epop_latest_text.txt', epop_text)
write_txt(f'{path}epop_prior_val.txt', str(round(final[-13], 1)))
write_txt(f'{path}epop_more_emp.txt', str(more_emp))
write_txt(f'{path}epop_less_emp.txt', str(less_emp))

#### EPOP bar range chart

save three csv files, one per educational group

In [62]:
HS_LTHS = ['LTHS', 'HS']
SC = ['SC']
COLL_ADV = ['COLL', 'ADV']

groups = [('Men 25-39', '25 <= AGE <=39 and FEMALE == 0 and EDUC in @HS_LTHS', 'HS_LTHS'),
          ('Men 40-54', '40 <= AGE <=54 and FEMALE == 0 and EDUC in @HS_LTHS', 'HS_LTHS'),
          ('Women 25-39', '25 <= AGE <=39 and FEMALE == 1 and EDUC in @HS_LTHS', 'HS_LTHS'),
          ('Women 40-54', '40 <= AGE <=54 and FEMALE == 1 and EDUC in @HS_LTHS', 'HS_LTHS'),
          ('Men 25-39', '25 <= AGE <=39 and FEMALE == 0 and EDUC in @SC', 'SC'),
          ('Men 40-54', '40 <= AGE <=54 and FEMALE == 0 and EDUC in @SC', 'SC'),
          ('Women 25-39', '25 <= AGE <=39 and FEMALE == 1 and EDUC in @SC', 'SC'),
          ('Women 40-54', '40 <= AGE <=54 and FEMALE == 1 and EDUC in @SC', 'SC'),
          ('Men 25-39', '25 <= AGE <=39 and FEMALE == 0 and EDUC in @COLL_ADV', 'COLL_ADV'),
          ('Men 40-54', '40 <= AGE <=54 and FEMALE == 0 and EDUC in @COLL_ADV', 'COLL_ADV'),
          ('Women 25-39', '25 <= AGE <=39 and FEMALE == 1 and EDUC in @COLL_ADV', 'COLL_ADV'),
          ('Women 40-54', '40 <= AGE <=54 and FEMALE == 1 and EDUC in @COLL_ADV', 'COLL_ADV')]

In [63]:
data = pd.DataFrame()
for year in year_range:
    year_data = pd.DataFrame()
    df = (pd.read_feather(f'cps{year}.ft')
            .filter(['HRMONTH', 'AGE', 'BASICWGT', 'EMP', 'FEMALE', 'EDUC'])
            .query('25 <= AGE <= 54'))
    for grp in groups:
        g = (df.query(grp[1])
               .groupby('HRMONTH')
               .apply(epop, 'EMP', 'BASICWGT'))
        g.index = pd.to_datetime([f'{year}-{i}-01' for i in g.index])
        grp_name = grp[0] + grp[2]
        year_data[grp_name] = g
    data = data.append(year_data)

In [64]:
combined_data = pd.DataFrame()
for grp in groups:
    grp_name = grp[0] + grp[2]
    d = data[grp_name]
    categories = [('max', d.max()-d.min()), ('min', d.min()),
                  ('12m', d.iloc[-13]), ('last', d.iloc[-1])]
    for cat in categories:
        combined_data.at[grp_name, cat[0]] = cat[1]

In [68]:
label = list(round(combined_data['last'] - combined_data['12m'], 1))
plus_minus = ['+' if val >= 0 else '-' for val in label]
last_val = list(combined_data['last'].round(1))
final = [f'{a}\% ({b}{c})' for a, b, c in zip(last_val, plus_minus, label)]

combined_data['Label'] = final

# Legend text
rng = '\scriptsize 1994-present range'
ltdt = data.index[-1].strftime('%b %Y')
prdt = data.index[-13].strftime('%b %Y')

legend = f'\legend{{{rng}, \scriptsize {prdt}, \scriptsize {ltdt}}}'

In [69]:
write_txt(f'{path}epop_grp_legend.txt', legend)

In [70]:
combined_data.to_csv(f'{path}epop_group.csv', index_label='group')